# Download a Network using a polygon

This notebook contains examples for the RA2CE tool. We will guide you through the basics of RA2CE : Configuring a road network from a geojson file from your local drive or from polygon coordinates.


First of all, we will import the packages we need to execute this notebook:

In [ ]:
from pathlib import Path
import geopandas as gpd
import folium 
from shapely.geometry import LineString, Polygon, box

Next we will call the necessary RA2CE scripts for our code

In [ ]:
import ra2ce.network.networks_utils as nut
from ra2ce.network.network_config_data.enums.network_type_enum import NetworkTypeEnum
from ra2ce.network.network_config_data.enums.road_type_enum import RoadTypeEnum
from ra2ce.network.network_config_data.network_config_data import (NetworkConfigData,NetworkSection,)
from ra2ce.network.network_wrappers.osm_network_wrapper.osm_network_wrapper import (OsmNetworkWrapper,)
from ra2ce.network.exporters.geodataframe_network_exporter import GeoDataFrameNetworkExporter
from ra2ce.network.exporters.multi_graph_network_exporter import MultiGraphNetworkExporter


## Folder structure

Before starting the examples, it is vital that you familiarize yourself with the RA2CE folder structure. RA2CE requires this folder structure to succesfully perform analyses. You can find the folder structure in the **documentation**.


A project folder must contain the subfolders 'output' and 'static'. An 'input' folder is optional for additional data sets but we will not use that in our example. It must also contain the network.ini and/or analyses.ini files. Within the subfolder 'static', RA2CE expects three subfolders: hazard, network, and output_graph. See below an example folder structure of “Project A”. This folder structure must be created by the user before running RA2CE.

Below you can add the path to your project directory to your RA2CE analysis

In [ ]:
# specify the name of the path to the project folder where you created the RA2CE folder setup
root_dir = Path('data','download_network_from_polygon')
assert root_dir.exists()

## Method A : Configuring a road network from OSM using a polygon

First we need to define the polygon. This should be a shapely polygon.
In this notebook, the polygon is defined through geojson coordinates. These coordinates you can copy paste from https://geojson.io/#map=2/0/20

### Defining a polygon

In [ ]:
geojson_data = {
  "type": "FeatureCollection",
  "features": [
    {
      "type": "Feature",
      "properties": {},
      "geometry": {
        "coordinates": [
          [
            [
              4.925796685034555,
              52.15567004009617
            ],
            [
              4.925796685034555,
              51.969875228118696
            ],
            [
              5.263478289905265,
              51.969875228118696
            ],
            [
              5.263478289905265,
              52.15567004009617
            ],
            [
              4.925796685034555,
              52.15567004009617
            ]
          ]
        ],
        "type": "Polygon"
      }
    }
  ]
}

minx, miny = geojson_data['features'][0]['geometry']['coordinates'][0][0]
maxx, maxy = geojson_data['features'][0]['geometry']['coordinates'][0][2]

polygon = box(minx, miny, maxx, maxy)


Let's explore the location of the polygon:

In [ ]:
gdf_polygon = gpd.GeoDataFrame(index=[0], crs='epsg:4326', geometry=[polygon])
gdf_polygon.explore(tiles="CartoDB positron")

### Set parameters and initalize a RA2CE run


Next we need to define the Network characteristics that needs to be downloaded

First we choose what kind of network that we need for the analysis. This is handled by the variable network_type in this notebook.
A network type can be :NONE/WALK/BIKE/DRIVE/DRIVE_SERVICE/ALL/INVALID 

The we can specify the resolution of the selected network by defining road_types.
Road types includes :NONE,MOTORWAY,MOTORWAY_LINK,TRUNK,TRUNK_LINK,PRIMARY,PRIMARY_LINK,SECONDARY,SECONDARY_LINK,TERTIARY,TERTIARY_LINK,RESIDENTIAL,ROAD,TUNNEL,BRIDGE,CULVERT,UNCLASSIFIED,INVALID

After that we can start accessing the data from OSM. 
Bear in mind that larger regions and / or the more detail included in the road network may increase download time (significantly)

In [ ]:

#First we define which roads we will want to download from OSM to create a network with
_network_section = NetworkSection(
    network_type=NetworkTypeEnum.DRIVE,
    road_types=[RoadTypeEnum.MOTORWAY,RoadTypeEnum.PRIMARY,RoadTypeEnum.ROAD], 
)

_network_config_data = NetworkConfigData(
    root_path= root_dir,
    network=_network_section,
    )

#pass the specified sections as arguments for configuration
_network_config_data = NetworkConfigData(network=_network_section, static_path=root_dir.joinpath('static'))
#download the network based on the polygon extent and the specified road characteristics
_graph,_gdf = OsmNetworkWrapper.get_network_from_polygon(_network_config_data, polygon)

### Explore the results

Lets explore the downloaded network

In [ ]:
m = _gdf.explore(column='highway', tiles="CartoDB positron")
folium.GeoJson(gdf_polygon,
               style_function=lambda x: {'color': 'grey', 'fillOpacity': 0.05, 'weight': 3},
               tooltip='polygon').add_to(m)

folium.LayerControl().add_to(m)
m

## Method B : Configuring a road network from OSM using a geojson file from a local drive

You can also use a geojson file in your local drive to to carry out the same process.

An easy way to use this option is by drawing a polygon at the location of choice at [geojson.io](http://geojson.io/). 
Another option is creating a polygon layer in a GIS and saving it as a *.geojson*.

The geojson polygon should be saved in the **static/network** folder.

In [ ]:
#Add the path to your Geojson file. 
_geojson_path = root_dir / 'static'/'network'/"map.geojson"

#visualize the polygon
gdf_polygon = gpd.read_file(_geojson_path)
gdf_polygon = gpd.GeoDataFrame(index=[0], crs='epsg:4326', geometry=[polygon])
gdf_polygon.explore(tiles="CartoDB positron")


### Set parameters and initalize a RA2CE run

Then you can specify the network characteristics the same way as defined in Method A

In [ ]:
#First we define which roads we will want to download from OSM to create a network with
_network_section = NetworkSection(
    polygon= Path(_geojson_path),
    network_type=NetworkTypeEnum.DRIVE,
    road_types=[RoadTypeEnum.MOTORWAY,RoadTypeEnum.PRIMARY,RoadTypeEnum.ROAD], 
    save_gpkg= True,
    
)

#pass the specified sections as arguments for configuration
_network_config_data = NetworkConfigData(
    root_path=root_dir,
    static_path=root_dir.joinpath('static'),
    network=_network_section,
    )

#download the network based on the polygon extent and the specified road characteristics
_graph,_gdf = OsmNetworkWrapper.get_network_from_geojson(_network_config_data)

### Explore the results

Lets explore the downloaded network

In [ ]:
a = _gdf.explore(column='highway', tiles="CartoDB positron")
folium.GeoJson(gpd.read_file(_geojson_path),
               style_function=lambda x: {'color': 'grey', 'fillOpacity': 0.05, 'weight': 3},
               tooltip='polygon').add_to(a)

folium.LayerControl().add_to(a)
a

## Finally we can save our downloaded network to the RA2CE folder setup

Finally, you can save the network that you downloaded through Method A or Method B to the output_graph folder in the RA2CE folder setup

In [ ]:
# Export the graph
_exporter = MultiGraphNetworkExporter(basename='base_graph', export_types=['gpkg', 'pickle'])
_exporter.export(export_path=root_dir.joinpath('static','output_graph'), export_data=_graph)

# Export the network
_exporter = GeoDataFrameNetworkExporter(basename='base_network', export_types=['gpkg', 'pickle'])
_exporter.export(export_path=root_dir.joinpath('static','output_graph'), export_data=_gdf)